In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import json

In [2]:
inDIR = './cnn_test_results/'
model_prefix = 'unetplus_*'

In [3]:
for idx, d in enumerate(glob.glob(os.path.join(inDIR, model_prefix))):
    inputs_tmp = '_'.join(os.path.basename(d).split('_')[1:])
    if idx == 0:
        df_bbox_gt = pd.read_csv(os.path.join(d, 'test_valid_bbox_gt.csv'))
        
        with open(os.path.join(d, 'test_valid_burrow.json'), 'r') as src:
            burrow_valid = json.load(src)
            burrow_valid['inputs'] = '_'.join(burrow_valid['inputs'])
            df_valid_burrow = pd.DataFrame(burrow_valid, index=[idx])
        
        with open(os.path.join(d, 'test_valid_pixel.json'), 'r') as src:
            pixel_valid = json.load(src)
            pixel_valid['inputs'] = '_'.join(pixel_valid['inputs'])
            df_valid_pixel = pd.DataFrame(pixel_valid, index=[idx])
            
        df_burrow_gt = pd.read_csv(os.path.join(d, 'test_valid_burrow_gt.csv'))
        df_burrow_ct = pd.read_csv(os.path.join(d, 'test_valid_burrow_count.csv'))
        df_burrow_ct.insert(0, 'inputs', inputs_tmp)
            
    else:
        df_bbox_gt_tmp = pd.read_csv(os.path.join(d, 'test_valid_bbox_gt.csv'))
        df_bbox_gt = pd.concat([df_bbox_gt, df_bbox_gt_tmp])
        
        with open(os.path.join(d, 'test_valid_burrow.json'), 'r') as src:
            burrow_valid = json.load(src)
            burrow_valid['inputs'] = '_'.join(burrow_valid['inputs'])
            df_valid_burrow_tmp = pd.DataFrame(burrow_valid, index=[idx])
            df_valid_burrow = pd.concat([df_valid_burrow, df_valid_burrow_tmp])
        
        with open(os.path.join(d, 'test_valid_pixel.json'), 'r') as src:
            pixel_valid = json.load(src)
            pixel_valid['inputs'] = '_'.join(pixel_valid['inputs'])
            df_valid_pixel_tmp = pd.DataFrame(pixel_valid, index=[idx])
            df_valid_pixel = pd.concat([df_valid_pixel, df_valid_pixel_tmp])
            
        df_burrow_gt_tmp = pd.read_csv(os.path.join(d, 'test_valid_burrow_gt.csv'))
        df_burrow_gt = pd.concat([df_burrow_gt, df_burrow_gt_tmp])
        
        df_burrow_ct_tmp = pd.read_csv(os.path.join(d, 'test_valid_burrow_count.csv'))
        df_burrow_ct_tmp.insert(0, 'inputs', inputs_tmp)
        df_burrow_ct = pd.concat([df_burrow_ct, df_burrow_ct_tmp])

In [4]:
df_valid_pixel = df_valid_pixel[['model', 'inputs', 'accuracy', 'precision', 'recall', 'fscore']]
df_valid_pixel.insert(2, 'n_inputs', df_valid_pixel['inputs'].apply(lambda x: len(x.split('_'))))
df_valid_burrow.insert(2, 'n_inputs', df_valid_burrow['inputs'].apply(lambda x: len(x.split('_'))))

In [24]:
df_valid_pixel.sort_values(['n_inputs', 'inputs']).round(2).drop(columns=['model'])

,inputs,n_inputs,accuracy,precision,recall,fscore
0,rgb,1,0.98,0.83,0.75,0.68
5,tpi,1,0.99,0.84,0.81,0.77
1,rgb_tpi,2,0.99,0.83,0.83,0.76
2,rgb_tpi_shade,3,0.99,0.82,0.84,0.77
3,rgb_tpi_shade_ndvi,4,0.99,0.85,0.83,0.78
4,rgb_tpi_shade_ndvi_dsm,5,0.99,0.86,0.83,0.78


In [25]:
df_valid_burrow.sort_values(['n_inputs', 'inputs']).round(2).drop(columns=['model'])

,inputs,n_inputs,precision,recall,fscore,corr
0,rgb,1,0.70,0.62,0.66,0.82
5,tpi,1,0.79,0.77,0.78,0.95
1,rgb_tpi,2,0.82,0.81,0.81,0.93
2,rgb_tpi_shade,3,0.75,0.86,0.80,0.91
3,rgb_tpi_shade_ndvi,4,0.78,0.85,0.81,0.94
4,rgb_tpi_shade_ndvi_dsm,5,0.78,0.82,0.80,0.92


In [7]:
df_burrow_ct.insert(1, 'n_inputs', df_burrow_ct['inputs'].apply(lambda x: len(x.split('_'))))

In [35]:
import scipy as sp
sns.set(font_scale = 1.3)
sns.set_style("ticks")
g = sns.lmplot(x='burrow_ct_gt', y='burrow_ct_pred', data=df_burrow_ct.sort_values(['n_inputs', 'inputs']),
               col='inputs', col_wrap=3, 
               scatter_kws={'color': 'black', 's': 10},
               line_kws={'color': 'black', 'linestyle': 'solid', 'linewidth': 1})

def annotate(data, **kws):
    r, p = sp.stats.pearsonr(data['burrow_ct_gt'], data['burrow_ct_pred'])
    mpe = np.mean((data['burrow_ct_pred'] - data['burrow_ct_gt']) / (data['burrow_ct_gt'].apply(lambda x: max(1, x))))
    ax = plt.gca()
    ax.text(.05, .8, 'r={:.2f}, p={:.2g}'.format(r, p),
            transform=ax.transAxes, fontsize=12)
    ax.text(.05, .74, 'MPE={:.2f}'.format(mpe),
            transform=ax.transAxes, fontsize=12)
    
g.map_dataframe(annotate)
for ax in g.axes:
    ax.plot(np.arange(0, 20), np.arange(0, 20), c='black', linestyle='dashed', linewidth=1)
    ax.set_ylabel('Burrow count (predicted)')
    ax.set_xlabel('Burrow count (observed)')
    ax.set_xticks(np.arange(0, 21, 5))
    ax.set_yticks(np.arange(0, 21, 5))

plt.savefig('./results_figs/burrow_ct_corr.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close()

In [36]:
df_burrow_gt['n_inputs'] = df_burrow_gt['inputs'].apply(lambda x: len(x.split('_')))

In [63]:
len(df_bbox_gt.Tile.unique())

32

In [67]:
len(df_burrow_gt[df_burrow_gt['is_burrow'] == 1].geometry.unique())

282

In [69]:
len(df_burrow_gt[(df_burrow_gt['is_burrow'] == 1) & (df_burrow_gt['activity'] > 0)].geometry.unique())

218

In [73]:
df_burrow_gt_recall = df_burrow_gt.groupby(['inputs', 'n_inputs']).apply(lambda x: ((x['is_burrow'] == 1) & (x['pred'] == 1)).sum() / 
                                                                       (x['is_burrow'] == 1).sum())

df_burrow_gt_recall.name = 'Recall'
df_burrow_gt_precision = df_burrow_gt.groupby(['inputs', 'n_inputs']).apply(lambda x: ((x['is_burrow'] == 1) & (x['pred'] == 1)).sum() / 
                                                                       (x['pred'] == 1).sum())

df_burrow_gt_precision.name = 'Precision'
df_burrow_gt_recall_a0 = df_burrow_gt[df_burrow_gt['activity'] != 0].groupby(['inputs', 'n_inputs']).apply(lambda x: ((x['is_burrow'] == 1) & (x['pred'] == 1)).sum() / 
                                                                       (x['is_burrow'] == 1).sum())#.reset_index(level).rename(columns={0: 'Recall'})

df_burrow_gt_recall_a0.name = 'Recall active'
df_burrow_gt_precision_a0 = df_burrow_gt[df_burrow_gt['activity'] != 0].groupby(['inputs', 'n_inputs']).apply(lambda x: ((x['is_burrow'] == 1) & (x['pred'] == 1)).sum() / 
                                                                       (x['pred'] == 1).sum())#.reset_index(level).rename(columns={0: 'Recall'})

df_burrow_gt_precision_a0.name = 'Precision active'

In [85]:
df_burrow_gt_smry = pd.merge(df_burrow_gt_precision, pd.merge(df_burrow_gt_recall,
                                       pd.merge(df_burrow_gt_precision_a0, df_burrow_gt_recall_a0, left_index=True, right_index=True),
                                       left_index=True, right_index=True),
         left_index=True, right_index=True).sort_values(['n_inputs', 'inputs'])
df_burrow_gt_smry.insert(2, 'F-score', (2 * df_burrow_gt_smry['Recall'] * df_burrow_gt_smry['Precision']) / (df_burrow_gt_smry['Recall'] + df_burrow_gt_smry['Precision']))
df_burrow_gt_smry['F-score active'] = (2 * df_burrow_gt_smry['Recall active'] * df_burrow_gt_smry['Precision active']) / \
                         (df_burrow_gt_smry['Recall active'] + df_burrow_gt_smry['Precision active'])

In [86]:
df_burrow_gt_smry.round(2)

,,Precision,Recall,F-score,Precision active,Recall active,F-score active
inputs,n_inputs,,,,,,
rgb,1,0.94,0.52,0.67,0.93,0.59,0.72
tpi,1,0.92,0.65,0.77,0.91,0.68,0.78
rgb_tpi,2,0.94,0.65,0.77,0.93,0.71,0.80
rgb_tpi_shade,3,0.93,0.73,0.82,0.91,0.77,0.84
rgb_tpi_shade_ndvi,4,0.94,0.70,0.80,0.93,0.74,0.82
rgb_tpi_shade_ndvi_dsm,5,0.94,0.68,0.79,0.92,0.73,0.82


In [42]:
df_burrow_gt_error = df_burrow_gt.groupby(['n_inputs', 'inputs']).apply(lambda x: ((x['is_burrow'] == 0) & (x['pred'] == 1)).sum() / (x['pred'] == 1).sum())
df_burrow_gt_error.name = 'Error rate'
df_burrow_gt_error = df_burrow_gt_error.reset_index().sort_values(['n_inputs', 'inputs'])
df_burrow_gt_error['Comment'] = 'Overall'
df_burrow_gt_error = pd.pivot(df_burrow_gt_error, index='Comment', columns=['n_inputs', 'inputs'], values='Error rate')
df_burrow_gt_error = df_burrow_gt_error.droplevel(0, axis=1)

In [43]:
df_burrow_gt_nb_error = df_burrow_gt[df_burrow_gt['is_burrow'] == 0].groupby(['inputs', 
                                                                              'n_inputs', 
                                                                              'Comment']).apply(lambda x: ((x['pred'] == 1)).sum() / len(x['pred']))
df_burrow_gt_nb_error.name = 'Error rate'
df_burrow_gt_nb_error = df_burrow_gt_nb_error.reset_index().sort_values(['n_inputs', 'inputs'])
df_burrow_gt_nb_error = pd.pivot(df_burrow_gt_nb_error, index='Comment', columns=['n_inputs', 'inputs'], values='Error rate')
df_burrow_gt_nb_error = df_burrow_gt_nb_error.droplevel(0, axis=1)
df_burrow_gt_nb_error.insert(0, 'n', df_burrow_gt[df_burrow_gt['is_burrow'] == 0]['Comment'].value_counts() // len(df_burrow_gt['inputs'].unique()))

In [44]:
df_burrow_gt_error.insert(0, 'n', df_burrow_gt_nb_error['n'].sum())

In [45]:
df_error_fnl = pd.concat([df_burrow_gt_nb_error, df_burrow_gt_error]).round(2)
df_error_fnl

inputs,n,rgb,tpi,rgb_tpi,rgb_tpi_shade,rgb_tpi_shade_ndvi,rgb_tpi_shade_ndvi_dsm
Comment,,,,,,,
ant,67,0.06,0.03,0.04,0.09,0.04,0.04
den,8,0.38,0.50,0.38,0.38,0.38,0.38
dig,6,0.17,0.17,0.17,0.50,0.33,0.33
old,27,0.04,0.30,0.15,0.11,0.15,0.19
Overall,108,0.06,0.08,0.06,0.07,0.06,0.06


In [61]:
plt.close()
from matplotlib.lines import Line2D
from scipy import stats
sns.set(font_scale = 1.3)
sns.set_style("ticks")
df_burrow_gt_std = df_burrow_gt[df_burrow_gt['is_burrow'] == 1].copy()
df_burrow_gt_std['height'] = stats.zscore(df_burrow_gt_std['height'])
df_burrow_gt_std['size'] = stats.zscore(df_burrow_gt_std['size'])
df_burrow_gt_std['activity'] = stats.zscore(df_burrow_gt_std['activity'])
df_burrow_gt_std['vegetation'] = stats.zscore(df_burrow_gt_std['vegetation'])
df_burrow_gt_long = pd.melt(df_burrow_gt_std,
                               id_vars=['n_inputs', 'inputs', 'Pasture', 'is_burrow', 'pred', 'label'],
                               value_vars=['height', 'size', 'activity', 'vegetation'])
#df_burrow_gt_long.loc[df_burrow_gt_long['variable'] == 'vegetation', 'value'] = 3.0 - df_burrow_gt_long.loc[df_burrow_gt_long['variable'] == 'vegetation', 'value']
g=sns.catplot(x='variable', y='value', data=df_burrow_gt_long.sort_values(['n_inputs', 'inputs']), kind='point',
                hue='pred', col='inputs', col_wrap=3,
                join=False, dodge=0.15, markers='o', scale=1.1, 
              palette={0.0: 'black', 1.0: 'black'}, ec=['black', 'none'], legend=False)
for ax in g.axes:
    #ax.set_ylim(0, 3)
    ax.legend().remove()
    p1=ax.get_children()[4]
    p1.set_facecolor('white')
    p1.set_linewidth(1)
    ax.set_xlabel('Burrow characteristic', fontsize=12)
    ax.set_ylabel('Observed value z-score', fontsize=12)
    ax.axhline(y=0, linestyle='dotted', color='black')

legend_elements = [Line2D([0], [0], marker='o', color='black', label='Missed',
                          markerfacecolor='white', markersize=10),
                  Line2D([0], [0], marker='o', color='black', label='Detected',
                          markerfacecolor='black', markersize=10)]

g.fig.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1.0, 0.0, 0.5, 0.5))
plt.savefig('./results_figs/missed_burrow_characteristics_zscore.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close()

In [51]:
df_bbox_gt = df_bbox_gt[df_bbox_gt['Type'] != 'non_burrows'].copy()

In [52]:
def val_metrics(df, prefix='pred'):
    #(group_label, df) = grouping
    precision = df[prefix + '_correct_ct'].sum() / df[prefix + '_ct'].sum()
    recall = df[prefix + '_correct_ct'].sum() / df['gt_ct'].sum()
    fscore = (2 * precision * recall) / (precision + recall)
    return pd.DataFrame({'Precision': precision,
                         'Recall': recall,
                         'F-score': fscore},
                        index=[len(df.name.split('_'))])
#df_bbox.groupby('inputs').apply(val_metrics)

In [53]:
df_bbox_gt_pred_smry = df_bbox_gt[df_bbox_gt['Digitize'] == 1].groupby('inputs').apply(val_metrics).reset_index().rename(columns={'level_1': 'n_inputs'}).sort_values('n_inputs')
df_bbox_gt_label_smry = df_bbox_gt[df_bbox_gt['Digitize'] == 1].groupby('inputs').apply(val_metrics, prefix='label').reset_index().rename(columns={'level_1': 'n_inputs'}).sort_values('n_inputs')

In [54]:
df_bbox_gt_label_smry.round(2)

,inputs,n_inputs,Precision,Recall,F-score
0,rgb,1,0.81,0.74,0.77
5,tpi,1,0.81,0.74,0.77
1,rgb_tpi,2,0.81,0.74,0.77
2,rgb_tpi_shade,3,0.81,0.74,0.77
3,rgb_tpi_shade_ndvi,4,0.81,0.74,0.77
4,rgb_tpi_shade_ndvi_dsm,5,0.81,0.74,0.77


In [55]:
len(df_bbox_gt[df_bbox_gt['Digitize'] == 1]['Tile'].unique())

13

In [56]:
df_bbox_gt_pred_smry.round(2)

,inputs,n_inputs,Precision,Recall,F-score
0,rgb,1,0.74,0.64,0.69
5,tpi,1,0.79,0.76,0.77
1,rgb_tpi,2,0.78,0.70,0.74
2,rgb_tpi_shade,3,0.72,0.78,0.75
3,rgb_tpi_shade_ndvi,4,0.75,0.77,0.76
4,rgb_tpi_shade_ndvi_dsm,5,0.70,0.76,0.73
